In [1]:
import numpy as np
from threading import Thread

In [2]:
A = np.genfromtxt("A.txt", delimiter=",")
b = np.genfromtxt("b.txt", delimiter=",")

## SLAE utils

In [3]:
def back_substitution(A, b, start_from="bottom"):
    if(start_from == "top"):
        return substitute_from_top(A, b)
    elif(start_from == "bottom"):
        return substitute_from_bottom(A, b)

In [4]:
def substitute_from_top(A, b):
    x = np.zeros_like(b)
    for i in range(0, A.shape[0]):
        x[i] = (b[i] - np.dot(A[i, 0:i], x[0:i])) / A[i][i]
    return x

In [5]:
def substitute_from_bottom(A, b):
    x = np.zeros_like(b)
    n = A.shape[0] - 1
    zero = -1
    for i in range(n, zero, -1):
        x[i] = (b[i] - np.dot(A[i, i+1:], x[i+1:])) / A[i,i]
    return x

In [6]:
def calculate_discrepancy(A, b, x):
    return np.dot(A, x) - b

In [7]:
def is_positive_defined(A):
    return np.all(np.linalg.eigvals(A) > 0)

In [8]:
def is_symmetric(A):
    return np.all(A==A.T)

In [9]:
A

array([[ 0.4974,  0.    , -0.1299,  0.0914,  0.1523],
       [-0.0305,  0.3248,  0.    , -0.0619,  0.0203],
       [ 0.0102, -0.0914,  0.5887,  0.0112,  0.0355],
       [ 0.0305,  0.    , -0.0741,  0.5887,  0.    ],
       [ 0.0203, -0.0305,  0.1472, -0.0122,  0.4263]])

In [10]:
b

array([ 1.5875, -1.759 ,  1.4139,  1.7702, -2.0767])

## Метод квадратного корня

In [11]:
def cholessky_decomposition(A):
    S = np.zeros_like(A)
    S[0,0] = np.sqrt(A[0,0])
    S[0,1:] = A[0,1:]/S[0,0]
    for i in range(1, S.shape[0]):
        S[i, i] = np.sqrt(np.abs(A[i, i] - np.sum(S[0:i, i]**2)))
        for j in range(i+1, S.shape[1]):
            S[i][j] = (A[i][j] - np.dot(S[0:i, i], S[0:i, j])) / S[i][i]
    return S

In [12]:
def solve(A, b):
    U = cholessky_decomposition(A)
    y = back_substitution(U.T, b, "top")
    x = back_substitution(U, y, "bottom")
    return x

In [13]:
%timeit solve(np.dot(A.T, A), b)

183 µs ± 9.36 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


## Метод встречной прогонки

## Метод отражений

In [14]:
class HausholderMethod:
    def __init__(self, A, b):
        self.A = np.copy(A)
        self.b = np.copy(b)
        self.n = self.b.shape[0]
        self.iteration = 0
        
    def decompose(self):
        for k in range(0, self.n-1):
            self.iteration = k
            s = self.get_s()
            e = self.get_e()
            alpha = self.calculate_alpha(s)
            omega = self.calculate_omega(alpha, s, e)
            A_ = np.copy(self.A[k:,k+1:])
            b_ = np.copy(self.b[k:])
            self.A[k][k] = alpha
            for i in range(k, self.n):
                self.b[i] = b_[i-k] - 2*omega[i]*np.dot(b_, omega[k:])
                for j in range(k+1, self.n):
                    self.A[i][j] = self.A[i][j] - 2*omega[i]*np.dot(A_[:,j-(k+1)], omega[k:])
            self.A[k+1:,k] = 0
    
    def solve(self):
        self.decompose()
        return back_substitution(self.A, self.b)
            
    def get_s(self):
        s = np.copy(self.A[:,self.iteration])
        s[:self.iteration] = 0
        return s
    
    def get_e(self):
        e = np.zeros(self.n)
        e[self.iteration] = 1
        return e
        
    def calculate_alpha(self, s):
        return np.linalg.norm(s)
    
    def calculate_x(self, alpha, s, e):
        return 1 / (np.sqrt(2*np.dot(s, s - alpha * e)))
    
    def calculate_omega(self, alpha, s, e):
        x = self.calculate_x(alpha, s, e)
        return x * (s - alpha*e)

In [15]:
x = HausholderMethod(A, b).solve()
x

array([ 4.9996129 , -3.99950458,  1.99890632,  2.99954266, -6.00005072])

## Метод простых итераций

In [16]:
class IterativeMethod:
    def __init__(self):
        pass
        
        
    def canonize(self, A, b):
        B = np.eye(*A.shape) - np.dot(A.T, A)/np.linalg.norm(np.dot(A.T, A))
        g = np.dot(A.T, b) / np.linalg.norm(np.dot(A.T, A))
        return B, g

    
    def is_converge(self, B):
        if np.linalg.norm(B) < 1:
            return "Method converges (||B|| is equal to {})".format(np.linalg.norm(B))
        if np.max(np.abs(np.linalg.eigvals(B))) < 1:
            return "Method converges (max eigenvalue is: {})".format(np.max(np.abs(np.linalg.eigvalsh(B))))
        return "Method doesn't converge"
    
    
    def estimate_iterations(self, B, g, eps):
        norm_B = np.linalg.norm(B)
        norm_g = np.linalg.norm(g)
        if(norm_B > 1):
            return "Impossible to estimate"
        return (np.log10(eps) + np.log10(1-norm_B) - np.log10(norm_g))/norm_g - 1
        
        
    def solve(self, A, b, x0=np.zeros_like(b), eps=1e-5, max_it=1e8, full_print=False):
        B, g = self.canonize(A, b)
        difference = 1
        it = 0
        x_prev = np.copy(x0)
        x_next = np.copy(x0)
        while(difference >= eps and it < max_it):
            x_next = np.dot(B, x_next) + g
            difference = np.linalg.norm(x_next - x_prev)
            x_prev = np.copy(x_next)
            it += 1
        discrepancy = calculate_discrepancy(A, b, x_prev)
        estimated_iterations = self.estimate_iterations(B, g, eps)
        is_method_converge = self.is_converge(B)
        if full_print:
            self.full_print(difference, it, x_prev, discrepancy, estimated_iterations, is_method_converge)
        return x_prev
    
    def full_print(self, difference, it, x_k, discrepancy, estimated_iterations, is_method_converge):
        print(is_method_converge)
        print("Estimated iterations: {0}\n".format(estimated_iterations))
        print("Error is: {0:.5E}\nNumber of iterations: {1}\n".format(difference, it))
        solution_str = ", ".join(["x[{0}] = {1:.5f}".format(i, x_i)
                                 for i, x_i in enumerate(x_k)])
        discrepancy_str = ", ".join(["eps[{0}] = {1:.5E}".format(i, eps_i)
                                    for i, eps_i in enumerate(discrepancy)])
        print("\nSolution is:\n" + solution_str)
        print("\nDiscrepancy is:\n" + discrepancy_str)

In [17]:
x = IterativeMethod().solve(A, b, max_it=1e15, full_print=True)

Method converges (max eigenvalue is: 0.8643513458921206)
Estimated iterations: Impossible to estimate

Error is: 8.66598E-06
Number of iterations: 69


Solution is:
x[0] = 4.99959, x[1] = -3.99954, x[2] = 1.99889, x[3] = 2.99954, x[4] = -6.00003

Discrepancy is:
eps[0] = -5.23166E-06, eps[1] = -1.17547E-05, eps[2] = -6.07765E-06, eps[3] = -3.25308E-07, eps[4] = 8.44158E-06


## Метод Гаусса-Зейделя

In [18]:
class GaussSeidelMethod:
    def __init__(self):
        pass
    
    
    def make_symmetric(self, A, b):
        return np.dot(A.T, A), np.dot(A.T, b)
    
    
    def is_converge(self, A):
        return is_symmetric(A) and is_positive_defined(A)
    
    
    def solve(self, A, b, x0=np.zeros_like(b), eps=1e-5, max_it=1e5, full_print=False):
        B, g = self.make_symmetric(A, b)
        difference = 1
        it = 0
        x_k = np.copy(x0)
        x_k_1 = np.copy(x0)
        while(difference > eps and it < max_it):
            x_k = np.copy(x_k_1)
            for i, _ in enumerate(x0):
                accumulator = 0
                accumulator += -np.dot(B[i, :i], x_k_1[:i]) - np.dot(B[i, i+1:], x_k[i+1:]) + g[i]
                x_k_1[i] = accumulator/B[i][i]
                it += 1
            difference = np.linalg.norm(x_k-x_k_1)
        discrepancy = calculate_discrepancy(A, b, x_k)
        is_system_converge = "Matrix converges" if self.is_converge(B) else "Matrix doesn't converge"
        if full_print:
            self.full_print(difference, it, x_k, discrepancy, is_system_converge)
        return x_k
    
    
    def full_print(self, difference, it, x_k, discrepancy, is_system_converge):
        print(is_system_converge)
        print("Error is: {0:.5E}\nNumber of iterations: {1}".format(difference, it))
        solution_str = ", ".join(["x[{0}] = {1:.5f}".format(i, x_i)
                                 for i, x_i in enumerate(x_k)])
        discrepancy_str = ", ".join(["eps[{0}] = {1:.5E}".format(i, eps_i)
                                    for i, eps_i in enumerate(discrepancy)])
        print("Solution is:\n" + solution_str)
        print("Discrepancy is:\n" + discrepancy_str)

In [19]:
x = GaussSeidelMethod().solve(A, b, full_print=True)

Matrix converges
Error is: 4.83646E-06
Number of iterations: 70
Solution is:
x[0] = 4.99961, x[1] = -3.99951, x[2] = 1.99890, x[3] = 2.99954, x[4] = -6.00005
Discrepancy is:
eps[0] = -1.78619E-06, eps[1] = -9.67277E-07, eps[2] = -1.12089E-06, eps[3] = 1.41424E-07, eps[4] = 7.77538E-07
